In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta

In [11]:
data = pd.read_csv("aapl_1h_train.csv")
rsi_data = ta.momentum.RSIIndicator(data.Close, window=14)
data["RSI"] = rsi_data.rsi()
data = data.dropna()
data

,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume,RSI
13,1602617400,0,2020-10-13 19:30:00,121.800003,122.050003,121.029998,121.050003,16221506.0,42.205490
14,1602682200,0,2020-10-14 13:30:00,121.000000,123.019996,120.040000,121.689903,41414544.0,46.975990
15,1602685800,0,2020-10-14 14:30:00,121.690002,122.389999,121.180000,121.599998,17702126.0,46.396536
16,1602689400,0,2020-10-14 15:30:00,121.610000,121.730003,119.709999,120.247703,22141570.0,38.669926
17,1602693000,0,2020-10-14 16:30:00,120.250000,120.769897,120.000000,120.139999,15375733.0,38.125346
...,...,...,...,...,...,...,...,...,...
390,1609432200,0,2020-12-31 16:30:00,132.529998,132.800003,131.720001,132.619995,13538948.0,40.278164
391,1609435800,0,2020-12-31 17:30:00,132.619995,132.717605,132.310104,132.560806,7474176.0,39.901949
392,1609439400,0,2020-12-31 18:30:00,132.565002,132.889999,132.009994,132.764999,9408857.0,41.917511
393,1609443000,0,2020-12-31 19:30:00,132.770004,133.179992,132.565994,132.800003,10147999.0,42.274917


In [12]:
def def_trading_strategy(data, n_shares, stop_loss_close, take_profit_close, Umbral):
    cash= 1_000_000
    com = 0.125/100
    history = []
    portfolio_value = []
    active_operations = []
    total_profit_loss = 0

    for i, row in data.iterrows():
        # Close active operation
        active_op_temp = []
        for operation in active_operations:
            if operation["stop_loss"] > row.Close:
                cash += (row.Close * operation["n_shares"]) * (1 - com)
                # Calcular la ganancia o pérdida y actualizar el total
                profit_loss = (row.Close * operation["n_shares"]) * (1 - com) - (operation["bought"] * operation["n_shares"])
                total_profit_loss += profit_loss
                history.append({"timestamp": row.Timestamp, "profit_loss": profit_loss})
            elif operation["take_profit"] < row.Close:
                cash += (row.Close * operation["n_shares"]) * (1 - com)
                # Calcular la ganancia o pérdida y actualizar el total
                profit_loss = (row.Close * operation["n_shares"]) * (1 - com) - (operation["bought"] * operation["n_shares"])
                total_profit_loss += profit_loss
                history.append({"timestamp": row.Timestamp, "profit_loss": profit_loss})
            else:
                active_op_temp.append(operation)
        active_operations = active_op_temp
            
        # ¿Tenemos suficiente efectivo?
        if cash < (row.Close * n_shares * (1 + com)):
            asset_vals = sum([operation["n_shares"] * row.Close for operation in active_operations])
            portfolio_value.append(cash + asset_vals)
            continue
        
        # Analizar la señal larga
        if row.RSI <= Umbral:  # If ceil(Op)
            active_operations.append({
                "timestamp": row.Timestamp,
                "bought": row.Close,
                "n_shares": n_shares,
                "type": "long",
                "stop_loss": row.Close * stop_loss_close,
                "take_profit": row.Close * take_profit_close
            })
            cash -= row.Close * n_shares * (1 + com)
        
        asset_vals = sum([operation["n_shares"] * row.Close for operation in active_operations])
        portfolio_value.append(cash + asset_vals)
    
    return total_profit_loss


In [13]:
import optuna

def objective(trial):
    # Define los rangos de búsqueda para los parámetros
    n_shares = trial.suggest_int('n_shares', 1, 50)
    stop_loss_close = trial.suggest_uniform('stop_loss_close', 0.80, 1)
    take_profit_close = trial.suggest_uniform('take_profit_close', 1, 2)
    Umbral = trial.suggest_uniform('Umbral', 0, 30)

    # Evalúa la función de trading con los parámetros sugeridos
    profit_loss = def_trading_strategy(data, n_shares, stop_loss_close, take_profit_close, Umbral)
    
    return profit_loss  # Devuelve la métrica que se debe minimizar/maximizar

# Crea un estudio Optuna
study = optuna.create_study(direction='maximize')  # 'maximize' si se quiere maximizar la métrica

# Ejecuta la optimización
study.optimize(objective, n_trials=20)  # Puedes ajustar n_trials según tus recursos y tiempo disponible

# Obtiene los mejores parámetros encontrados
best_params = study.best_params
best_profit_loss = study.best_value

print("Mejores parámetros encontrados:")
print(best_params)
print("Rendimiento (Profit/Loss):", best_profit_loss)


[I 2024-02-25 19:11:20,910] A new study created in memory with name: no-name-162fdcd5-bd84-4fb7-8384-719ab16fb388
C:\Users\elias\AppData\Local\Temp\ipykernel_16972\2141988989.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  stop_loss_close = trial.suggest_uniform('stop_loss_close', 0.80, 1)
C:\Users\elias\AppData\Local\Temp\ipykernel_16972\2141988989.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  take_profit_close = trial.suggest_uniform('take_profit_close', 1, 2)
C:\Users\elias\AppData\Local\Temp\ipykernel_16972\2141988989.py:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use sugges

Mejores parámetros encontrados:
{'n_shares': 39, 'stop_loss_close': 0.8236547790231283, 'take_profit_close': 1.187668531160739, 'Umbral': 28.62734217908081}
Rendimiento (Profit/Loss): 9330.535887660004


In [14]:
# Condiciones para comprar y vender
condicion_compra = data["RSI"] <= 30
condicion_venta = data["RSI"] > 70

# Aplicando las condiciones para obtener señales de compra y venta
data['Senal_RSI'] = ''
data.loc[condicion_compra, 'Senal_RSI'] = 'Compra'
data.loc[condicion_venta, 'Senal_RSI'] = 'Venta'
data.to_excel('RSI_BOS.xlsx', index=False)

In [15]:
data['Senal_RSI'].value_counts()

Senal_RSI
          330
Venta      23
Compra     14
Name: count, dtype: int64